In [1]:
import numpy
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')
import pycountry
sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua
import numba
import copy
import glob
from numba import njit
import pandas
import glob

In [2]:
def request(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test/')
    z.close()
    
    files = glob.glob('./test/*.nc')
    if len(files) > 0:
        ds = xarray.open_dataset(files[0])            
        to_be_removed = ['trajectory_index', 'trajectory']
        for ivar in list(ds.variables):
            if 'string' in ivar:
                to_be_removed.append(ivar)
            
            if 'trajectory' in ds[ivar].dims and ivar not in list(ds.coords):
                report_id = ds[ivar].astype(object).sum(axis=1).astype(str)
                ds = ds.drop_vars(ivar)
                ds[ivar] = ('obs', report_id.values[ds.trajectory_index.values])  # todo obs ???
                
            if ds[ivar].ndim > 1:
                tmp= ds[ivar].astype(object).sum(axis=1).astype(str)
                ds = ds.drop_vars(ivar)
                idim = tmp.dims[0]
                ds[ivar] = (idim, tmp)

        ds = ds.drop_vars(to_be_removed)
        data = ds.to_dataframe()
    else:
        files = glob.glob('./test/*.csv')
        data = pd.read_csv(files[0])
    
    os.remove('download.zip')
    try:
       shutil.rmtree('./test/')
    except:
       print('Error while deleting directory')
    
    return data

In [3]:
request({
            'variable': 'relative_humidity',
            'statid': '94910',
            'date': '20100101-20100201',
        })

2021-01-27 11:27:44,525 INFO Welcome to the CDS
2021-01-27 11:27:44,528 INFO Sending request to https://sis-dev.climate.copernicus.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2021-01-27 11:27:44,768 INFO Request is completed
2021-01-27 11:27:44,769 INFO Downloading http://136.156.132.176/cache-compute-0000/cache/data1/adaptor.comprehensive_upper_air.retrieve-1611670230.1464458-25809-2-36ac72ef-c3d6-426f-a1d6-5c421feb20c4.zip to download.zip (39.1K)
2021-01-27 11:27:44,892 INFO Download rate 322.6K/s


,hur,lat,lon,plev,time,trajectory_label
obs,,,,,,
0,0.010662,-35.169998,147.449997,1620.0,2009-12-31 23:00:00,10000040371
0,0.011324,-35.169998,147.449997,1920.0,2009-12-31 23:00:00,10000040371
0,0.008908,-35.169998,147.449997,2000.0,2009-12-31 23:00:00,10000040371
0,0.012713,-35.169998,147.449997,2560.0,2009-12-31 23:00:00,10000040371
0,0.008358,-35.169998,147.449997,3000.0,2009-12-31 23:00:00,10000040371
...,...,...,...,...,...,...
0,0.677614,-35.169998,147.449997,85000.0,2010-01-31 23:00:00,10000040433
0,0.682122,-35.169998,147.449997,88900.0,2010-01-31 23:00:00,10000040433
0,0.640196,-35.169998,147.449997,90300.0,2010-01-31 23:00:00,10000040433
